In [47]:
import PyPDF2
import re
from IPython.display import display, Markdown

import os
import openai

import json
credentials = json.load(open('keys.json'))
openai.organization = credentials['organization_key']
openai.api_key = credentials['api_key']

In [2]:
class Section:
    def __init__(self, number, title, text, lo, gpt_notes = ""):
        self.number = number
        self.title = title
        self.text = text
        self.lo = lo
        self.gpt_notes = gpt_notes

    def __str__(self):
        lo_string = "\n\n".join(f'*{lo}*' for lo in self.lo)
        return f'**{self.number} {self.title}**\n\n{lo_string}\n\n{self.gpt_notes}'
    
    def display(self):
        display(Markdown(str(self)))

In [3]:
def get_pdf_content(file_path):
    pdf_file_obj = open(file_path, 'rb')
    reader = PyPDF2.PdfReader(pdf_file_obj)

    content = ""
    for page_num in range(len(reader.pages)):
        page_obj = reader.pages[page_num]
        content += page_obj.extract_text()

    pdf_file_obj.close()

    return content

def find_sections(content):
    pattern = r"(7\.\d+(\.\d+)*)(.*?)(?=(7\.\d+(\.\d+)*)|$)"
    sections_raw = re.findall(pattern, content, re.DOTALL)
    sections = []
    for t in sections_raw:
        number = t[0]
        title = t[2].split("\n")[0].lstrip().rstrip()
        text = t[2]
        text = text.replace(title, '').replace("This element addresses the following learning objective of this course:\n", '').replace("This element addresses the following learning objectives of this course:", '').replace("\n", " ").replace("● ", "").replace("  ", " ").lstrip().rstrip()
        text_without_lo = remove_lo(text)
        lo = get_lo(text)
        sections.append(Section(number, title, text_without_lo, lo))
    return sections

def get_lo(content):
    pattern = r"(LO\d+:.*?[.!?])"
    los = re.findall(pattern, content)
    return [re.sub(r'LO\d+: ', '', lo) for lo in los]

def remove_lo(content):
    pattern = r"(LO\d+:.*?[.!?])"
    return re.sub(pattern, '', content, flags=re.MULTILINE)

In [4]:
# Read in the PDF
pdf_content = get_pdf_content('Week 7 transcript  (updated Feb 22).pdf')

In [5]:
# Overwrite all the workd that is done. DO NOT DO THIS UNLESS YOU WANT TO START OVER
sections = find_sections(pdf_content)

In [6]:
for section in sections:
    if section.text == "" or "on the following prompt" in section.text:
        continue
    if section.gpt_notes != "":
        continue
    else:
        gpt_content = f"""Topic: {section.title}
        Detail Level: I would like something between high-level summary and detailed notes.
        Learning Objectives: {'; '.join(section.lo)}
        Format: I prefer the notes to be in bullet-point or table format.
        Course Transcript: {section.text}
        Purpose: The notes are for my personal understanding and to facilitate in class discussion with my peers."""
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", 
                 "content": gpt_content}
                 ])
        section.gpt_notes = completion.choices[0].message.content

    print(section.title, "- Complete")

Data Collection Philosophy Complete
Existing or New Data Complete
Acquire and Clean Data Complete
Overview of Methods Complete
Tips on How to Choose a Method Complete
Internal and External Validity Complete
The Intersection of Data, Method, and Complete
Introduction to a Perspective from IBM’s Journey Complete
 Complete
 Complete
 Complete
Design Research to Generate Applied Insight Complete
Keep the Audience in Mind Complete
Communicate Insight, Hypotheses Testing, and the Word “Prove” Complete
Different Deliverables Complete
Different Audiences Complete
Creswell and Creswell, Chapter 10: Mixed Methods Procedures Complete
The Data Detective by T im Harford Complete


In [39]:
for section in sections:
    if section.gpt_notes == "":
        # print(section.title)
        display(Markdown('**' + str(section.title) + '**'))
    else:
        section.display()

**Data Collection**

**7.1.1 Data Collection Philosophy**

*Assess and select specific data and the data collection methods that best fit a specific outcome or need.*

- Earlier philosophy: collect as much data as possible and worry about what to do with it later
- Current philosophy: collect as much data as you need and not more
- Strategic collection of data can minimize unnecessary risk
- Collect data with purpose to avoid paying for unnecessary server space
- International and regional policies on data protection and privacy affect data collection philosophy
- Assess and select data collection methods that best fit specific outcome or need
- Collecting too much data requires storage and analysis of larger quantities of data
- Purposeful data collection helps avoid sifting through unnecessary data 
- Collecting only necessary data can help ensure compliance with regulations on data protection and privacy

**7.1.2 Existing or New Data**

*Assess and select specific data and the data collection methods that best fit a specific outcome or need.*

- Deciding between using existing or new data
- Trade-offs between off-the-shelf data and new data collection
- Consider the purpose of the initial data collection when deciding if existing data is sufficient
- Example of when existing data may be sufficient: auto manufacturer collects data on production and output, and wants to test a new piece of equipment for efficiency. Leadership wants to deploy it in the lines with the most potential for impact, even if it's in lower yield production lines.
- Example of when existing data may not be sufficient: major retailer has data on curbside pickup popularity but wants to consider the viability of home delivery. Can't automatically extrapolate and should consider collecting new data about users' willingness to embrace home delivery or using industry data.
- Be aware of the initial intent of why the data was collected and articulate how existing data will help answer the question.

**7.1.3 Acquire and Clean Data**

*Assess and select specific data and the data collection methods that best fit a specific outcome or need.*

Acquiring and Cleaning Data:

Acquiring Data:
- Identify the data needed for a specific outcome or need
- Determine the best data collection method (Surveys, interviews, experiments, observations, etc.)
- Consider the quality and reliability of the data source
- Obtain permission to access necessary data

Cleaning Data:
- Identify missing or incomplete data
- Remove duplicate data
- Standardize data formats (e.g., date and time formats)
- Remove outliers and errors
- Transform data to ensure the data is consistent with analysis objectives
- Ensure data accuracy and completeness
- Label variables clearly to make it easier to work with in the subsequent analysis

Assessing Data Quality:
- Completeness: Is the data missing any values or variables?
- Accuracy: How precise is the data?
- Validity: Is the data relevant and appropriate for the analysis?
- Consistency: Are there any discrepancies or errors in the data?
- Timeliness: Is the data up-to-date and relevant to the current analysis?

Overall, acquiring and cleaning data can be a time-consuming process, but it is essential to obtain reliable and usable data for analysis. It is crucial to identify specific data needs, select the best data collection method, and assess the data quality to ensure the data is accurate, complete, and valid.

**The Intent of Data Collection**

...

**7.7.2 Different Deliverables**

*Justify an analytic approach that informs decision making.

Identify the audience and the most ef fective method to communicate a persuasive argument.*

- Final deliverable will depend on the audience and objective of research
- Possible deliverables:
  - Slide deck and presentation
  - Paper or report
  - Proof of concept or minimum viable product
  - Code-based deliverables
  - New model
- Important to future-proof work, consider future self or how someone else may interpret code
- Be sure to annotate to aid future understanding

**7.7.3 Different Audiences**

*Justify an analytic approach that informs decision making.

Identify the audience and the most ef fective method to communicate a persuasive argument.*

- When communicating, it's important to remember who your audience is
- The intended audience should be a guiding heuristic for communication
- The audience will impact the way you communicate in terms of vocabulary and emphasis on project aspects
- The level of technical expertise will also impact communication style
- Incentive structure is another factor to consider - the C-suite may care more about high-level recommendations and narrative compared to a technical team who may be convinced by nuances
- Justifying an analytic approach should be tailored to the audience and what will be persuasive to them
- Identifying the most effective method to communicate a persuasive argument is crucial for decision making.

**Creswell and Creswell Textbook**

**7.8.1 Creswell and Creswell, Chapter 10: Mixed Methods Procedures**

*Design and apply research questions.

Assess and select data and the data collection methods that best fit a specific outcome or need.

Justify an analytic approach that informs decision-making.*

Mixed Methods Procedures - Creswell and Creswell, Chapter 10:
- Mixed methods is a combination of quantitative and qualitative research methods to address research problem
- It is important to ensure that the purpose of the study and the purpose of the mixed method approach is made clear
- There are three high-level approaches to mixed method research:
  - Quantitative and qualitative are done simultaneously to see if the results converge
  - Qualitative work informs quantitative work by identifying the relationships to be studied
  - Quantitative study is done first to identify interesting patterns, followed by qualitative approach to explain the quantitative results
- Worldview, or our view about science and how the world works, can influence problem definition, the questions we ask, and which findings we emphasize
- Acknowledging and being mindful of our worldview throughout the project can help us avoid biases and be more open-minded
- Awareness of our biases can help us slow down our thinking and consider alternative perspectives that cannot be easily measured.

**7.9.1 The Data Detective by T im Harford**

*Design and apply research questions.

Assess and select data and the data collection methods that best fit a specific outcome or need.

Justify an analytic approach that informs decision-making.*

- Tim Harford's book "The Data Detective" emphasizes the importance of approaching statistics with a healthy level of skepticism and curiosity.
- The introduction shows a contrast with "How to Lie with Statistics" but doesn't promote fear of statistics.
- Rule 6: Ask Who is Missing encourages thinking about who is missing from data sets and studies, including gender and racial minorities.
- It's important to consider both sample error and sample bias in analyzing data.
- The Golden Rule: Be Curious emphasizes the value of curiosity, embracing failure, and storytelling in the research and decision-making process.
- "Tell me more" summarizes the book's invitation to learn more and embrace a healthy skepticism and curiosity.